
### Prediction based anomaly detection


This is a slightly modified clone of the Kyle Hundman's github repo [telemanom](https://github.com/khundman/telemanom).
See also the article [Detecting Spacecraft Anomalies Using LSTMs and Nonparametric Dynamic Thresholding](https://www.groundai.com/project/detecting-spacecraft-anomalies-using-lstms-and-nonparametric-dynamic-thresholding/#bib.bib14) in groundai.com.

The approach has two components, an LSTM-NN for prediction and a sophisticated thresholding approach to actually detect anomalies. (The authors compared their approach with Gaussian Tail approaches such as z-score in one dimension or elliptic envelopes for multiple dimensions and showed their approach yields better results.)

The LSTM part is quite similar to the approach chosen for [LSTM Neural Networks for Anomaly Detection](https://medium.com/datadriveninvestor/lstm-neural-networks-for-anomaly-detection-4328cb9b6e27)


In [1]:
import os
import logging
import threading
import pandas as pd  
import math
import dill
import numpy as np  
import matplotlib.pyplot as plt
import pydot
import seaborn as seabornInstance
import networkx
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import anomaly
from iotfunctions import bif
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator, ui, base, bif
from iotfunctions.base import BaseTransformer
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.enginelog import EngineLogging
from iotfunctions import pipeline as pp
from iotfunctions.stages import DataWriterSqlAlchemy, DataWriterFile
from iotfunctions.pipeline import JobController, DataAggregator

import datetime as dt
from scipy.linalg import norm
import scipy as sp
from scipy import signal
from scipy import linalg
from pyod.models.cblof import CBLOF
from pyod.models.ocsvm import OCSVM
from pyod.models.xgbod import XGBOD

#import pmdarima as pm
import inverse_covariance as icov # from package skggm
from inverse_covariance import QuicGraphLasso

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

from skimage import util as skiutil # for nifty windowing

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


from keras.models import Sequential
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import LSTM, Dense, Flatten, Input, Dropout, Convolution1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate
from keras.utils import plot_model
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from keras.utils import model_to_dot
import telemanom
from telemanom.helpers import Config
from telemanom.errors import Errors
import telemanom.helpers as helpers
from telemanom.channel import Channel
from telemanom.modeling import Model

%matplotlib inline

/home/markus/.local/lib/python3.7/site-packages/iotfunctions/bif.py:1605: UserWarning: IoTCalcSettings is deprecated. Use entity type constants instead of a metadata provider to set entity type properties
  warnings.warn(('IoTCalcSettings is deprecated. Use entity type constants'
/home/markus/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/markus/.local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised wh

In [2]:
# load original data set
#!curl -O https://s3-us-west-2.amazonaws.com/telemanom/data.zip && unzip data.zip -d telemanom && rm data.zip
!mkdir -p ./telemanom/data/2020-01-29_15.00.10/data

In [3]:
# set up the default model parameters
# hidden layers 	2
# units in hidden layers 	80
# sequence length (ls) 	250
# training iterations 	30
# dropout 	0.3
# batch size 	64
# optimizer 	Adam
logger = helpers.setup_logging()
logger.setLevel(logging.DEBUG)
conf = Config("/home/markus/src/mmfunctions/telemanom/config.yaml")

In [4]:
# Load data from 
device="E-3"
chan = Channel(conf, device)
helpers.make_dirs(conf.use_id, conf, "./telemanom")

In [5]:
chan.load_data("./telemanom")
# chan.train
df2 = pd.DataFrame(chan.train)
df2

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-0.247183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.247183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.247183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.247183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.247183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,0.086224,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2876,0.169473,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2877,0.169473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2878,0.169473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# split training data into windows of size 260: 
#   lookback aka l_s (defined in config.yaml as 250) + n_predictions (defined in config.yaml as 10)
#   to predict the last 10 data points from the past 250 ones
#   this is quite similar to our windowsize parameter !
chan.shape_data(chan.train, train=True)

In [7]:
print(chan)


Channel:Channel
Training data shape: (2620, 250, 25), (2620, 10)
  Test data shape: (8047, 250, 25), (8047, 10)
  Original data shape: (2880, 25), (8307, 25)


In [14]:
# T-9,MSL,"[[780, 810], [890, 970]]","[point, point]",1096
model = Model(conf, conf.use_id, chan, "./telemanom", False)

In [15]:
model.train_new(chan)

Train on 2096 samples, validate on 524 samples
Epoch 1/35
2096/2096 [==============================] - 8s 4ms/step - loss: 0.0773 - val_loss: 0.0186
Epoch 2/35
2096/2096 [==============================] - 8s 4ms/step - loss: 0.0203 - val_loss: 0.0092
Epoch 3/35
2096/2096 [==============================] - 8s 4ms/step - loss: 0.0131 - val_loss: 0.0059
Epoch 4/35
2096/2096 [==============================] - 8s 4ms/step - loss: 0.0115 - val_loss: 0.0056
Epoch 5/35
2096/2096 [==============================] - 8s 4ms/step - loss: 0.0097 - val_loss: 0.0043
Epoch 6/35
2096/2096 [==============================] - 8s 4ms/step - loss: 0.0086 - val_loss: 0.0030
Epoch 7/35
2096/2096 [==============================] - 8s 4ms/step - loss: 0.0077 - val_loss: 0.0030
Epoch 8/35
2096/2096 [==============================] - 8s 4ms/step - loss: 0.0071 - val_loss: 0.0031
Epoch 9/35
2096/2096 [==============================] - 8s 4ms/step - loss: 0.0071 - val_loss: 0.0028
Epoch 10/35
2096/2096 [============

In [16]:
print (model)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, None, 80)          33920     
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 80)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 80)                51520     
_________________________________________________________________
dropout_6 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                810       
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total params: 86,250
Trainable params: 86,250
Non-trainable params: 0
__________________________________________________

In [17]:
df = pd.DataFrame(chan.X_train[0])
df.describe()
# P-10,MSL,"[[4590, 4720]]",[point],6100
#df.iloc[2750:2800]

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
count,250.000000,250.000000,250.0,250.000000,250.0,250.000000,250.000000,250.0,250.0,250.0,...,250.0,250.0,250.000000,250.000000,250.000000,250.0,250.000000,250.000000,250.0,250.0
mean,0.295121,0.024000,0.0,0.024000,0.0,0.148000,0.116000,0.0,0.0,0.0,...,0.0,0.0,0.024000,0.024000,0.012000,0.0,0.028000,0.028000,0.0,0.0
std,0.238559,0.153356,0.0,0.153356,0.0,0.355812,0.320867,0.0,0.0,0.0,...,0.0,0.0,0.153356,0.153356,0.109104,0.0,0.165304,0.165304,0.0,0.0
min,-0.080396,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
25%,0.086224,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
50%,0.335858,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
75%,0.502095,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
max,0.834159,1.000000,0.0,1.000000,0.0,1.000000,1.000000,0.0,0.0,0.0,...,0.0,0.0,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.0,0.0


In [24]:
model.save("./telemanom")

In [18]:
# print (chan.X_train)
model.batch_predict(chan, Path="./telemanom")
# predict test data batch-wise and append the results

In [27]:
print (chan.y_hat.shape, chan.y_test.shape)

(8047,) (8047, 10)


In [21]:
# smooth the prediction error and apply exponential weights to it
errors = Errors(chan, conf, conf.use_id, "./telemanom")

#  for each overlapping window establish a threshold so that removing error points above it 
# maximizes the reduction of mean and standard deviation. Sort of an adaptive z-score 
errors.process_batches(chan)

normalized prediction error: 0.02


In [28]:
print (errors.E_seq, " \n ", errors.anom_scores)

[(5570, 5709)]  
  [{'start_idx': 5320, 'end_idx': 5389, 'score': 3.504268339986817}, {'start_idx': 5390, 'end_idx': 5459, 'score': 3.7353472227877074}]
